# ETL Pipeline- Working Copy

## Imports

In [1]:
# Std lib:
import os
import warnings

# Querying data:
from google.cloud import bigquery

# Data manipulation:
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon

# Visualization:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
style.use('seaborn')

# Display all columns in Jupyter:
from IPython.display import display
pd.options.display.max_columns = None

# Filter Warnings
warnings.filterwarnings('ignore')

## Configure credentials

In [2]:
key_location = # Your key location here (remove before commit). Maybe we can use os to configure this automatically?
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_location

## Set up client

In [3]:
client = bigquery.Client()
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("crime")
table = client.get_table(table_ref)

## Query the Crime Table

In [4]:
# ENTER YOUR QUERY HERE:

MAX_GB = 2 # Change this if desired

QUERY = """
SELECT *
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year in (2016)
"""

In [5]:
def safe_query_to_dataframe(client, table, sql_query, max_gb=0):
    """
    Wrapper function for bigquery.client.query.  Will throw an error if the query exceeds the desired limit.
        
    params
        > client: a bigquery client object
        > table: a bigquery table object
        > sql_query (string): an SQL query on the table
        > max_gb (int): GB limit of query
        
    returns
        > Error: if query size exceeds limit
        > Dataframe: Dataframe representation of the query
    """
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    query_job = client.query(sql_query, job_config=job_config)
    gbs_used = query_job.total_bytes_processed / 1e9
    assert gbs_used < max_gb, f"This query will process {gbs_used} GB, which exceeds your desired limit of {max_gb} GB."
    query_job = client.query(sql_query)
    return query_job.to_dataframe()

In [6]:
df = safe_query_to_dataframe(client=client, table=table, sql_query=QUERY, max_gb=MAX_GB)

## Pipeline

In [7]:
# We can remove objects (ie. rows) with any null values since we have huge dataset with millions of rows
def remove_null_objects(df):
    """
    Removes objects with any null values -> less than 3000 rows dropped
    """
    df.dropna(inplace = True)
    
    
def add_dt_attributes(df):
    """
    Adds datetime attributes for easier querying
    """
    df["month"] = pd.to_numeric(df.date.dt.month, downcast="unsigned")
    df["hour"] = pd.to_numeric(df.date.dt.hour, downcast="unsigned")
    df["dayofweek"] = pd.to_numeric(df.date.dt.dayofweek, downcast="unsigned")
    df["weekend"] = df.date.dt.dayofweek >= 5
    

def remove_irrelevant_attributes(df): 
    """
    Remove attributes not needed for EDA or data mining 
    """
    df.drop(columns = ["unique_key", "case_number", "updated_on"], inplace = True)
    

def add_fbi_code_description(df):
    """
    Add fbi_code_description attribute for details and querying
    """
    df["fbi_code_description"] = np.where(df.fbi_code.str.startswith("01"), "Murder", 
                                 np.where(df.fbi_code.str.startswith("02"), "Rape", 
                                 np.where(df.fbi_code.str.startswith("03"), "Robbery",
                                 np.where(df.fbi_code.str.startswith("04"), "Assault",
                                 np.where(df.fbi_code.str.startswith("05"), "Burglary",
                                 np.where(df.fbi_code.str.startswith("06"), "Theft",
                                 np.where(df.fbi_code.str.startswith("07"), "Auto Theft",
                                 np.where(df.fbi_code.str.startswith("09"), "Forgery",
                                 np.where(df.fbi_code.str.startswith("10"), "Fraud/Embezzlement",
                                 np.where(df.fbi_code.str.startswith("11"), "Shots Fired",
                                 np.where(df.fbi_code.str.startswith("13"), "Prostitution",
                                 np.where(df.fbi_code.str.startswith("14"), "Indecent Exposure",
                                 np.where(df.fbi_code.str.startswith("15"), "Domestic Violence",
                                 np.where(df.fbi_code.str.startswith("16"), "Narcotics",
                                 np.where(df.fbi_code.str.startswith("17"), "Violation of liquor laws",
                                 np.where(df.fbi_code.str.startswith("18"), "Intoxicated subject",
                                 np.where(df.fbi_code.str.startswith("19"), "Disorderly conduct",
                                 np.where(df.fbi_code.str.startswith("20"), "Vagrants",
                                 np.where(df.fbi_code.str.startswith("21"), "Gambling",
                                 np.where(df.fbi_code.str.startswith("22"), "DWI",
                                 np.where(df.fbi_code.str.startswith("23"), "Reckless driving",
                                 np.where(df.fbi_code.str.startswith("24"), "Suspicious vehicle/person",
                                 np.where(df.fbi_code.str.startswith("26"), "All others: arson/vandalism",
                                 np.where(df.fbi_code.str.startswith("27"), "Are you in trouble",
                                 np.where(df.fbi_code.str.startswith("28"), "Frequent patrol",
                                 np.where(df.fbi_code.str.startswith("29"), "Dog bite",
                                 np.where(df.fbi_code.str.startswith("30"), "Suicide",
                                 np.where(df.fbi_code.str.startswith("31"), "Miscellaneous deaths",
                                 np.where(df.fbi_code.str.startswith("32"), "Nature call",
                                 np.where(df.fbi_code.str.startswith("33"), "Welfare check",
                                 np.where(df.fbi_code.str.startswith("34"), "Affray in progress",
                                 np.where(df.fbi_code.str.startswith("35"), "Alarm call",
                                 np.where(df.fbi_code.str.startswith("40"), "En-route to district",
                                 np.where(df.fbi_code.str.startswith("41"), "En-route to home", "Unidentified"))))))))))))))))))))))))))))))))))
    df["fbi_code_description"] = df["fbi_code_description"].astype("category")
                                                                     

def convert_coords_to_geometry(df):
    """
    Converts lat/lon attributes to more usable geometry objects
    """
    df['geometry'] = list(zip(df.longitude, df.latitude))
    df['geometry'] = df['geometry'].apply(Point)
    df = geopandas.GeoDataFrame(df, geometry='geometry')
    
    

def join_with_econ(df):
    """
    Joins crime data with revised_econ data
    """
    econ_df = pd.read_excel("../data/cfnai/cfnai-realtime-revised.xlsx")
    econ_df['month'] = econ_df.Month.dt.month
    econ_df['month'] = pd.to_numeric(econ_df.month, downcast="unsigned")
    econ_df['year'] = econ_df.Month.dt.year
    econ_df.drop("Month", inplace=True, axis=1)
    return pd.merge(df, econ_df) # Can't do this inplace :(


def etl_pipeline(df):
    """
    This is the whole pipeline.  
    Add function calls to mutate the inputted dataframe into something that we can work with.
    """
    remove_null_objects(df)
    add_dt_attributes(df)
    remove_irrelevant_attributes(df)
    add_fbi_code_description(df)
    convert_coords_to_geometry(df)
    df = join_with_econ(df)
    return df

## Check out the Results

### Original

In [8]:
df.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11414017,JB392763,2016-04-01 19:35:00+00:00,098XX S DREXEL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,511,5,8,50,11,NaN,NaN,2016,2018-08-15 16:09:09+00:00,NaN,NaN,None
1,11516992,JB528700,2016-12-27 08:40:00+00:00,034XX S PAULINA ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,None,False,False,912,9,12,59,11,NaN,NaN,2016,2018-11-26 16:16:31+00:00,NaN,NaN,None
2,10399601,HZ136401,2016-02-01 06:30:00+00:00,089XX S ESSEX AVE,0930,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,STREET,False,False,423,4,7,48,07,1194388.0,1846033.0,2016,2018-02-10 15:50:01+00:00,41.732442,-87.563454,"(41.732442272, -87.563454259)"
3,10421405,HZ159871,2016-02-21 12:30:00+00:00,058XX W AINSLIE ST,0520,ASSAULT,AGGRAVATED:KNIFE/CUTTING INSTR,APARTMENT,False,True,1622,16,45,11,04A,1136222.0,1931975.0,2016,2018-02-10 15:50:01+00:00,41.969510,-87.774495,"(41.969510097, -87.774495484)"
4,10501453,HZ242608,2016-04-27 17:30:00+00:00,034XX W FOSTER AVE,1360,CRIMINAL TRESPASS,TO VEHICLE,VEHICLE NON-COMMERCIAL,False,False,1712,17,39,13,26,1152695.0,1934339.0,2016,2018-02-10 15:50:01+00:00,41.975686,-87.713861,"(41.975686228, -87.713860671)"


#### Check for missing data to resolve them

In [9]:
# There are null values in location_description, x_coordinate, y_coordinate, latitude, longitude and location
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269432 entries, 0 to 269431
Data columns (total 22 columns):
unique_key              269432 non-null int64
case_number             269432 non-null object
date                    269432 non-null datetime64[ns, UTC]
block                   269432 non-null object
iucr                    269432 non-null object
primary_type            269432 non-null object
description             269432 non-null object
location_description    268213 non-null object
arrest                  269432 non-null bool
domestic                269432 non-null bool
beat                    269432 non-null int64
district                269432 non-null int64
ward                    269432 non-null int64
community_area          269432 non-null int64
fbi_code                269432 non-null object
x_coordinate            267230 non-null float64
y_coordinate            267230 non-null float64
year                    269432 non-null int64
updated_on              269432 non-nul

In [10]:
# A lot of unknown location_descriptions
df[df.location_description.isnull()].head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11414017,JB392763,2016-04-01 19:35:00+00:00,098XX S DREXEL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,511,5,8,50,11,NaN,NaN,2016,2018-08-15 16:09:09+00:00,NaN,NaN,None
1,11516992,JB528700,2016-12-27 08:40:00+00:00,034XX S PAULINA ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,None,False,False,912,9,12,59,11,NaN,NaN,2016,2018-11-26 16:16:31+00:00,NaN,NaN,None
18,11640660,JC206579,2016-03-14 21:15:00+00:00,040XX W 115TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,2211,22,19,74,11,NaN,NaN,2016,2019-04-01 16:21:13+00:00,NaN,NaN,None
57,11737822,JC323818,2016-01-01 23:25:00+00:00,091XX S GREENWOOD AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,413,4,8,47,11,NaN,NaN,2016,2019-06-28 16:26:10+00:00,NaN,NaN,None
119,10483861,HZ222691,2016-04-11 15:35:00+00:00,029XX N HARLEM AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,2511,25,36,18,11,1127578.0,1918682.0,2016,2018-02-10 15:50:01+00:00,41.933183,-87.806581,"(41.933182891, -87.806580645)"


In [11]:
# x_coordinate, y_coordinate, latitude, longitude and location have null values all on same objects
df[df.x_coordinate.isnull()].head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11414017,JB392763,2016-04-01 19:35:00+00:00,098XX S DREXEL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,511,5,8,50,11,NaN,NaN,2016,2018-08-15 16:09:09+00:00,NaN,NaN,None
1,11516992,JB528700,2016-12-27 08:40:00+00:00,034XX S PAULINA ST,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,None,False,False,912,9,12,59,11,NaN,NaN,2016,2018-11-26 16:16:31+00:00,NaN,NaN,None
18,11640660,JC206579,2016-03-14 21:15:00+00:00,040XX W 115TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,2211,22,19,74,11,NaN,NaN,2016,2019-04-01 16:21:13+00:00,NaN,NaN,None
57,11737822,JC323818,2016-01-01 23:25:00+00:00,091XX S GREENWOOD AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,413,4,8,47,11,NaN,NaN,2016,2019-06-28 16:26:10+00:00,NaN,NaN,None
138,11315578,JB264128,2016-02-19 00:00:00+00:00,130XX S ELLIS AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,533,5,9,54,11,NaN,NaN,2016,2018-05-15 15:55:49+00:00,NaN,NaN,None


#### Check for redundant data to resolve them

In [12]:
# Great! No duplicate objects
df.duplicated().sum()

0

### Pipelined

In [13]:
cp = df.copy()

In [14]:
cp = etl_pipeline(cp);

In [15]:
cp.head()

,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,latitude,longitude,location,month,hour,dayofweek,weekend,fbi_code_description,geometry,CF,CF3,PI,EUH,CH,SOI
0,2016-02-01 06:30:00+00:00,089XX S ESSEX AVE,0930,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,STREET,False,False,423,4,7,48,07,1194388.0,1846033.0,2016,41.732442,-87.563454,"(41.732442272, -87.563454259)",2,6,0,False,Auto Theft,POINT (-87.56345 41.73244),-0.261093,-0.14071,-0.27907,-0.015423,0.027763,0.005636
1,2016-02-21 12:30:00+00:00,058XX W AINSLIE ST,0520,ASSAULT,AGGRAVATED:KNIFE/CUTTING INSTR,APARTMENT,False,True,1622,16,45,11,04A,1136222.0,1931975.0,2016,41.969510,-87.774495,"(41.969510097, -87.774495484)",2,12,6,True,Assault,POINT (-87.77450 41.96951),-0.261093,-0.14071,-0.27907,-0.015423,0.027763,0.005636
2,2016-02-05 12:39:00+00:00,043XX W 55TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,ATM (AUTOMATIC TELLER MACHINE),False,False,813,8,13,62,11,1148159.0,1867777.0,2016,41.793121,-87.732256,"(41.793121443, -87.732255805)",2,12,4,False,Shots Fired,POINT (-87.73226 41.79312),-0.261093,-0.14071,-0.27907,-0.015423,0.027763,0.005636
3,2016-02-28 20:00:00+00:00,060XX S KEATING AVE,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,RESIDENCE-GARAGE,False,False,813,8,13,64,14,1145739.0,1864091.0,2016,41.783053,-87.741223,"(41.783052547, -87.741222911)",2,20,6,True,Indecent Exposure,POINT (-87.74122 41.78305),-0.261093,-0.14071,-0.27907,-0.015423,0.027763,0.005636
4,2016-02-07 13:00:00+00:00,063XX S CICERO AVE,2851,PUBLIC PEACE VIOLATION,ARSON THREAT,STREET,False,False,813,8,13,64,26,1145464.0,1862238.0,2016,41.777973,-87.742278,"(41.777972797, -87.742277863)",2,13,6,True,All others: arson/vandalism,POINT (-87.74228 41.77797),-0.261093,-0.14071,-0.27907,-0.015423,0.027763,0.005636


In [16]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266452 entries, 0 to 266451
Data columns (total 31 columns):
date                    266452 non-null datetime64[ns, UTC]
block                   266452 non-null object
iucr                    266452 non-null object
primary_type            266452 non-null object
description             266452 non-null object
location_description    266452 non-null object
arrest                  266452 non-null bool
domestic                266452 non-null bool
beat                    266452 non-null int64
district                266452 non-null int64
ward                    266452 non-null int64
community_area          266452 non-null int64
fbi_code                266452 non-null object
x_coordinate            266452 non-null float64
y_coordinate            266452 non-null float64
year                    266452 non-null int64
latitude                266452 non-null float64
longitude               266452 non-null float64
location                266452 non-

In [230]:
# TODO: export the cleaned data to a csv